In [ ]:
import sqlite3
import hashlib
import unittest

#teste
def hash_password(password):
    """Hash a senha usando SHA-256."""
    return hashlib.sha256(password.encode()).hexdigest()

def create_user(conn, username, password):
    """Cria um novo usuário no banco de dados."""
    cursor = conn.cursor()
    cursor.execute('INSERT INTO usuarios (username, password) VALUES (?, ?)', 
                   (username, hash_password(password)))
    conn.commit()

def authenticate_user(conn, username, password):
    """Verifica se as credenciais do usuário estão corretas."""
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM usuarios WHERE username = ? AND password = ?', 
                   (username, hash_password(password)))
    return cursor.fetchone() is not None



class TestLoginSenha(unittest.TestCase):
    def setUp(self):
        # Cria banco temporário em memória
        self.conn = sqlite3.connect(':memory:')
        self.conn.execute('''CREATE TABLE usuarios (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE,
            password TEXT
        )''')

    def tearDown(self):
        self.conn.close()

    def test_usuario_e_senha_correta(self):
        create_user(self.conn, "usuario1", "senha123")
        self.assertTrue(authenticate_user(self.conn, "usuario1", "senha123"))

    def test_senha_incorreta(self):
        create_user(self.conn, "usuario2", "senha_certa")
        self.assertFalse(authenticate_user(self.conn, "usuario2", "senha_errada"))

    def test_usuario_inexistente(self):
        self.assertFalse(authenticate_user(self.conn, "fantasma", "123456"))

    def test_hash_igual_para_mesma_senha(self):
        hash1 = hash_password("abc123")
        hash2 = hash_password("abc123")
        self.assertEqual(hash1, hash2)

    def test_hash_diferente_para_senhas_diferentes(self):
        hash1 = hash_password("senha1")
        hash2 = hash_password("senha2")
        self.assertNotEqual(hash1, hash2)



unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestLoginSenha))